In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Read data
df = pd.read_csv('data.csv')
# 쓸데없는 index 제거
df = df.drop(df.columns[0], axis=1)
print(df.head())

train_hours = pd.to_datetime(df['_time'])
print(train_hours.tail(15))

# moduleName = cosmos만 고르기
cosmos_df = df[df['chainName'] == "cosmos"]
columns_to_keep = ['_time', 'price', 'totalVolume', 'bondedTokens', 'tps']
cosmos_df = cosmos_df[columns_to_keep]
print(cosmos_df.head())

cols = list(cosmos_df)[1:5]
print(cols)

df_for_training = df[cols].astype(float)

# data normalizing
scaler = MinMaxScaler()
scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

trainX = []
trainY = []

n_future = 1
n_past = 24 # 24시간

for i in range(n_past, len(df_for_training_scaled) - n_future + 1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training_scaled.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])
    
trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

# ...

## 여기 shape 수정해야될 거 같은데.
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

# fit the model
history = model.fit(trainX, trainY, epochs=102, batch_size=24, validation_split=0.2, verbose=1, callbacks=[early_stopping])

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

                       _time chainName    ath                   athDate   atl   
0  2023-01-24 21:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16  \
1  2023-01-24 22:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   
2  2023-01-24 23:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   
3  2023-01-25 00:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   
4  2023-01-25 01:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   

                    atlDate         cap  capChange24h  capChangeRatio24H   
0  2020-03-13T02:27:44.591Z  3826578089     -69338669           -1.77978  \
1  2020-03-13T02:27:44.591Z  3834398759     -44719269           -1.15282   
2  2020-03-13T02:27:44.591Z  3813167577     -31946650           -0.83084   
3  2020-03-13T02:27:44.591Z  3806684761     -14779955           -0.38676   
4  2020-03-13T02:27:44.591Z  3820293895       3705855            0.09710   

   capRank  ...  gov-boxplot  not_bonded_tokens_pool-box